In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier
%matplotlib inline

In [28]:
df = pd.read_csv('../../Datasets/Social_Network_Ads.csv')

df.drop('User ID', axis=1, inplace=True)
df.head(5)

,Gender,Age,EstimatedSalary,Purchased
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0


In [29]:
df.Gender = pd.get_dummies(df.Gender, drop_first=True)
X = df.to_numpy()

np.random.seed = 0
X = X[np.random.permutation(X.shape[0])]

y = X[:, -1]
X = X[:, :-1]

In [30]:
split = int(X.shape[0] * 0.8)

X_train = X[:split]
y_train = y[:split]

X_test = X[split:]
y_test = y[split:]

In [31]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [57]:
clfs = [('LR', LogisticRegression()),
        ('KNN3', KNeighborsClassifier(n_neighbors=3)),
        ('KNN5', KNeighborsClassifier(n_neighbors=5)),
        ('KNN7', KNeighborsClassifier(n_neighbors=7)),
        ('KNN9', KNeighborsClassifier(n_neighbors=9)),
        ('SVM_l', SVC(kernel='linear')),
        ('SVM_r', SVC(kernel='rbf')),
        ('DT2', DecisionTreeClassifier(max_depth=2)),
        ('DT3', DecisionTreeClassifier(max_depth=3))]

In [48]:
for name, clf in clfs:
    clf.fit(X_train, y_train)
    print('{}:\ttrain_acc:{},\ttest_acc:{}'.format(
        name,
        clf.score(X_train, y_train),
        clf.score(X_test, y_test),
        ))

LR:	train_acc:0.84375,	test_acc:0.8625
KNN:	train_acc:0.915625,	test_acc:0.9375
SVM:	train_acc:0.90625,	test_acc:0.9125
DT:	train_acc:0.909375,	test_acc:0.925


/home/mqod/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [52]:
en_clf = VotingClassifier(clfs, n_jobs=-1)
en_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('KNN',
                              KNeighborsClassifier(algorithm='auto',
                                                   leaf_size=30,
                                                   metric='minkowski',
                                                   metric_params=None,
                                                   n_jobs=None, n_neighbors=5,
                                                   p=2, weights='uniform')),
                             ('SVM',
                              SVC(C=1.0, cache_size=200, class_weight=None,
                                  coef0=0.0, decision_function_shape='ovr',
                                  degree=3, gamma='auto_deprecated',
                                  kernel='rbf', max_iter=-1, probability=False,
                                  random_st...
                                  verbose=False)),
                             ('DT',
                              DecisionTreeClassifier(class_

In [54]:
print(en_clf.score(X_train, y_train))
print(en_clf.score(X_test, y_test))

0.909375
0.925
